In [ ]:
# @title TSD-WebUI Kurulum ve Yapılandırma
import os,subprocess,sys,fileinput,time,requests,json,subprocess,gc,threading,time,re,socket,urllib.request,ipywidgets as widgets
from datetime import datetime
from subprocess import getoutput
from IPython.utils import capture
from PIL import Image
from IPython.display import clear_output,display, HTML, Image
from io import BytesIO
import zipfile, random, string
!pip install python-multipart

# Varsayılan yapılandırma değerleri
check_value = {
    'DriveSyn' : True,
    'Setting_Preset' : "None",
    'Lib' : True,
    'FastMode': True,
    'root_folder' :'/content',
    'User_folder' : '/content/drive/MyDrive/TSD-Data',
    'API_folder': '/content/TSD-WebUI',
    'TSDFolder': '/content/TSD-WebUI',
    'Version':'ComfyUI',
    'OptionMode':'base',
    'Controlnet':'none',
    'SDmodel':'',
    'SD15model':'',
    'SDXLmodel':'',
    'Fluxmodel':'',
    'TSDmodel':'',
    'InpaintMd':'',
    'DriveLib' : True,
    'CommandLine' : '',
    'frontend_folder': '/usr/local/lib/python3.12/dist-packages/comfyui_frontend_package/static',
}

if 'UI_Version' in globals():
  Version = UI_Version
  
for key, value in check_value.items():
  if key not in globals():
    globals()[key] = value
  elif type(globals()[key]) == str:
    globals()[key] = globals()[key].split(' ')[-1]

def replace_text_file(file_path, old_text, new_text):
      """Dosya içindeki metni değiştir"""
      with open(file_path, 'r', encoding='utf-8') as file:
          file_content = file.read()
      updated_content = file_content.replace(old_text, new_text)
      with open(file_path, 'w', encoding='utf-8') as file:
          file.write(updated_content)

def inf(msg, style, wdth):
    """Bilgi mesajı göster"""
    inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(inf)
    
def aria_down(link,path,name):
    """Aria2 ile dosya indir"""
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d  {path} -o {name}

def c_folder(path):
    """Klasör oluştur"""
    path = f'{User_folder}/{path}'
    os.makedirs(path, exist_ok=True)

def default_file_check(folder_source,folder_check,over=False):
    """Varsayılan dosyaları kontrol et ve kopyala"""
    for item in os.listdir(folder_source):
        file_path = os.path.join(folder_source, item)
        if os.path.isfile(file_path):
            cp_path = file_path.replace(TSDFolder, folder_check)
            !cp {'-f' if over else '-n'} {file_path} {cp_path}   
        elif os.path.isdir(file_path):
            default_file_check(file_path,folder_check,over)

def civit_downlink(link):
    """CivitAI linkini indirme linkine çevir"""
    !wget {link} -q -O model.html
    try:
        with open('model.html', 'r', encoding='utf-8') as file:
            html_content = file.read()
        pattern = r'"modelVersionId":(\d+),'
        model_id = re.findall(pattern, html_content)
        if model_id:
            api_link = f'https://civitai.com/api/download/models/{model_id[0]}'
            print(f'Model indirme linki: {api_link}')
            return api_link
        else:
            return "Model ID bulunamadı."
    except requests.RequestException as e:
        return f"Sayfa yüklenirken hata: {e}"

def check_link(link):
    """Link formatını kontrol et ve düzenle"""
    if 'huggingface.co' in link:
        if 'blob' in link:
            link = link.replace('blob', 'resolve')
            return link
        else:
            return link
    if 'civitai.com' in link:
        if 'civitai.com/models' in link:
            return civit_downlink(link)
        else:
            return link
    
def token(link):
    """Link için token ekle"""
    if "civitai" in link:
        token = f'?token=8c7337ac0c39fe4133ae19a3d65b806f'
    else:
        token = ""
    return token
    
def run_list_txt(txt):
    """Txt dosyasındaki komutları çalıştır"""
    with open(f"{txt}", "r") as file:
        lists = file.readlines()
    for list in lists :
        !{list}

def link_folder(source,target):
    """Sembolik bağlantı oluştur"""
    !rm -rf {target}
    !ln -s {source} {target}

def check(path):
    """Klasör yoksa oluştur"""
    if not os.path.exists(path):
        os.makedirs(path)

def split_string_at(link):
    """Link ve dosya adını ayır"""
    delimiter = "@="
    if delimiter in link:
        parts = link.split(delimiter)
        if '.ckpt' not in parts[1] and '.gguf' not in parts[1] and '.safetensors' not in parts[1]:
            parts[1] = f'{parts[1]}.safetensors'
        return parts[0], parts[1]
    else:
        return link, None
        
def download(link):
    """Model dosyasını indir"""
    downloadcode = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
    link = link.replace('&', '\&')
    valuelink, valuename = split_string_at(link)
    valuelink = valuelink.split("?")[0]
    valuelink = check_link(valuelink)
    if '#' not in link:
        if valuename != None :
            if 'civit' not in link:
                !{downloadcode} {valuelink} -d {checkpointpatch} -o {valuename}
            else:
                !wget {valuelink}{token(valuelink)} -O {checkpointpatch}/{valuename}
        else:
            !wget {valuelink}{token(valuelink)} -P {checkpointpatch} --content-disposition

def download_controlnet(link):
    """ControlNet modelini indir"""
    downloadcode = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
    valuename = link.split('/')[-1].split('?')[0]
    !{downloadcode} {link} -d {controlnetpath} -o {valuename}
    
def download_lib(model,modellist):
    """Kütüphaneden model indir"""
    downloadcode = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
    if 'https:' in model:
        download(model)
    else:
        if not any(ext in model for ext in ['.ckpt', '.gguf', '.safetensors']):
            model += '.safetensors'
        if model in modellist:
            !{downloadcode} {modellist[model]} -d {checkpointpatch} -o {model}

def download_txt(lines):
    """Txt listesindeki modelleri indir"""
    for line in lines:
        download(line)

def download_txt_controlnet(txt_name):
    """ControlNet txt listesini indir"""
    if '/' in txt_name:
        file_path = txt_name
    else:
        file_path = f'{controlnetpath}/{txt_name}'
    with open(file_path, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]
    for line in lines:
        if '#' not in line:
            download_controlnet(line)

def defaultGraph(my_default_name, defaultGraph):
    """Varsayılan workflow'u ayarla"""
    if not os.path.isdir(frontend_folder):
        return
    default_path = f'{TSDFolder}/ComfySetting/{defaultGraph}'
    with open(default_path, 'r', encoding='utf-8') as js_file:
        default = js_file.read()

    my_default_path = f'{TSDFolder}/ComfySetting/{my_default_name}'
    with open(my_default_path, 'r', encoding='utf-8') as js_file:
        my_default = js_file.read()

    index_path = f'{frontend_folder}/index.html'
    with open(index_path, 'r', encoding='utf-8') as html_file:
        html_content = html_file.read()

    pattern = r'<script type="module" crossorigin src="./([^"]+)"></script>'
    module_path = re.findall(pattern, html_content)
    module_path[0]

    module_path_full = f'{frontend_folder}/{module_path[0]}'
    with open(module_path_full, 'r', encoding='utf-8') as file:
        file_content = file.read()
    updated_content = file_content.replace(default, my_default)
    with open(module_path_full, 'w', encoding='utf-8') as file:
        file.write(updated_content)

# Tunnel sunucu işlevleri
def check_cloudflare_setting():
    """Cloudflare ayarlarını kontrol et"""
    domain_setting = f"{User_folder}/Setting/Domain_sever.txt"
    with open(domain_setting, "r", encoding="utf-8") as file:
        lines = file.readlines()
    output = [None, None]
    for line in lines:
        if "cloudflare" in line and "#" not in line:
            flare_token = line.split("-")[-1].strip()
            flare_domain = line.split("-")[1].strip()
            output = [flare_token, flare_domain]
    return output
            
def cloudflare_thread(port):
    """Cloudflare tunnel başlat"""
    import socket
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    flare_token, flare_domain = check_cloudflare_setting()
    if flare_token != None:
        subprocess.Popen(["cloudflared", "tunnel", "run", "--token", flare_token])
        print(f"\033[92m{'🔗 Çevrimiçi kullanım linki (özel cloudflare):'} \033[0m", f"https://{flare_domain}")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print(f"\033[92m{'🔗 Çevrimiçi kullanım linki:'} \033[0m", l[l.find("http"):], end='')
            html="<div><code style='color:yellow'>Hızlı başlangıç modunda çalışıyorsunuz. Bu mod daha az node yükler ve controlnet/ipadapter gibi eklentileri önceden yüklemez. Tam özellikli kullanım için FastMode'u kapatın veya farklı Preset seçin ve bölüm 1'i tekrar çalıştırın</code></div>"
            if FastMode:
                display(HTML(html))

def localtunnel_thread(port):
    """LocalTunnel başlat"""
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nLocalTunnel sunucusu başlatılıyor\n")
    print("LocalTunnel şifresi:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
        print(line.decode(), end='')

def serveo_thread(port):
    """Serveo tunnel başlat"""
    import socket
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    try:
        process = subprocess.Popen(
            ["ssh", "-o", "StrictHostKeyChecking=no", "-R", "80:localhost:{}".format(port), "serveo.net"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        for line in iter(process.stdout.readline, ''):
            match = re.search(r'(https?://[^\s]+)', line)
            if match:
                print(f"\033[92m{'🔗 Çevrimiçi kullanım linki:'} \033[0m", match.group(1))
                break
    except Exception as e:
        print(f"❌ Hata: {e}")

def tunnelto_thread(port,api):
    """TunnelTo başlat"""
    import socket
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    cmd = ["/root/.tunnelto/bin/tunnelto", "set-auth", "--key", api[0]]
    subprocess.run(cmd)
    cmd_run = ["/root/.tunnelto/bin/tunnelto", "--subdomain", api[1], "--port", f"{port}"]
    process = subprocess.Popen(cmd_run)
    print(f"\033[92m{'🔗 Çevrimiçi kullanım linki:'} \033[0m", f"https://{api[1]}.tunn.dev")

def sever_flare(port, api = None):
    """Tunnel sunucularını başlat"""
    if api != None:
        threading.Thread(target=tunnelto_thread, daemon=True, args=(port,api,)).start()
    threading.Thread(target=cloudflare_thread, daemon=True, args=(port,)).start()
    threading.Thread(target=serveo_thread, daemon=True, args=(port,)).start()

# Çalıştırma işlevleri
def run_comfy_background(final_arg):
    """ComfyUI'yi arka planda çalıştır"""
    !pkill -f 'main.py'
    os.system(f"python main.py {final_arg} &")

def install_app(appfolder):
    """Uygulama yükle"""
    !pip install -q -r {appfolder}/requirements.txt
    !cp -rf {appfolder}/example {User_folder}/ComfyUIinput
    default_wf(appfolder)
    %cd {Version_folder}/custom_nodes
    with open(f"{appfolder}/node.txt", "r") as file:
        notes = file.readlines()
    for note in notes :
        last_segment = note.rsplit('/', 1)[-1].strip()
        if not os.path.exists(f'{last_segment }'):
            !git clone {note}
            !pip install -q -r {Version_folder}/custom_nodes/{last_segment}/requirements.txt
            !ln -s {appfolder}/prompt.dat /content/ComfyUI/prompt.dat
            !ln -s {appfolder}/workflow.json /content/ComfyUI/styles.csv
    %cd /content/ComfyUI

def default_wf(appfolder):
    """Varsayılan workflow ayarla"""
    json_path = f"{appfolder}/workflow.json"
    with open(json_path, "r", encoding="utf-8") as file:
        json_data = json.load(file)
    js_content = f"const defaultGraph = {json.dumps(json_data, indent=2)};"
    js_path = f"{TSDFolder}/ComfySetting/workflow.js"
    with open(js_path, "w", encoding="utf-8") as file:
        file.write(js_content)
    defaultGraph("workflow.js", "sd_defaultGraph.js")

def random_app(parent_folder):
    """Rastgele uygulama klasörü oluştur"""
    subfolders = [f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))]
    if not subfolders:
        return None
    folder_main = os.path.join(parent_folder, random.choice(subfolders))
    folder_name = ''.join(random.choices(string.ascii_letters, k=5))
    r = os.path.join(folder_main, folder_name)
    os.makedirs(r, exist_ok=True)
    return r

def extract_app(App_name,Ver):
    """Uygulama arşivini çıkart"""
    global API_folder
    API_folder = random_app("/content/ComfyUI/comfy")
    host = "http://turkish-ai.github.io/store/"
    download_link = f"{host}{App_name}/{Ver}.zip"
    !wget -q {download_link} -O {API_folder}/{Ver}.zip
    zip_path = f"{API_folder}/{Ver}.zip"
    extract_to = f"{API_folder}"
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)
    install_app(API_folder)

def run_ver(Version = "ComfyUI", CommandLine = "", tunnelto = None, Run = "None"):
    """Sürümü çalıştır"""
    %cd {Version_folder}
    
    # TunnelTo taraması
    if Run != 'None' or "App_name" in globals():
        CommandLine = "--disable-metadata"
    if tunnelto == None:
        domain_setting = f"{User_folder}/Setting/Domain_sever.txt"
        with open(domain_setting, "r", encoding="utf-8") as file:
            lines = file.readlines()
        for line in lines:
            if "tunn.dev" in line and "#" not in line:
                tunnelto = [line.split("-")[1].strip(),line.split("-")[0].split(".tunn.dev")[0].strip()]
                print(f"[{tunnelto[0]},{tunnelto[1]}]")
                break
                
    if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2":
        sever_flare(7860, tunnelto)
        drive_arg = f"--ckpt-dir {User_folder}/Model --embeddings-dir {User_folder}/Embeddings" if DriveLib else ""
        final_arg = f"{''if FastMode else '--xformers'} --theme dark --enable-insecure-extension-access --disable-safe-unpickle --share --no-half-vae {CommandLine} \\
                    --ui-settings-file {User_folder}/Setting/WebUI_Setting.json \\
                    --styles-file {User_folder}/Setting/WebUI_Styles.csv --lora-dir {User_folder}/Lora {drive_arg}"
        !python launch.py {final_arg}
        
    if Version == "ComfyUI":
        sever_flare(8888, tunnelto)
        final_arg = f"--preview-method auto --port 8888 --input-directory {User_folder}/ComfyUIinput {CommandLine}"
        if Run == 'None':
            if "App_name" in globals() :
                if API_folder == '/content/TSD-WebUI':
                    extract_app(App_name,Ver)
                    clear_output()
                run_comfy_background(final_arg)
                !python {API_folder}/app.py --share
            else:
                !python main.py {final_arg}
        else:
            run_comfy_background(final_arg)
            !python {API_folder}/API/{Run}/app.py --share
            
    if Version == "Kohya" :
        html="<p style='color:yellow'><code><b>Eğitim sırasında görüntülenme sorunu nedeniyle, komut satırında şu komutu çalıştırarak daha iyi yönetim yapabilirsiniz:<b></code></p><h3 style='color:lightgreen'><code>[ cd KohyaUI;python kohya_gui.py --share --headless ]</code></b></h3>"
        display(HTML(html))
        !python {Version_folder}/kohya_gui.py --share --headless
        
    if Version == "Fooocus" :
        !python {Version_folder}/entry_with_update.py --share --theme dark --disable-preset-download {CommandLine}
        
    if Version == "AutoRetouch":
        !python {Version_folder}/auto_retoucher.py
        
    if Version == "FluxGym":
        %run {TSDFolder}/Run_Fluxgym.ipynb

def xformers_check():
    """XFormers sürümünü kontrol et"""
    xformersver = {
        'Forge-v2':'0.0.27.post2',
        'Automatic': '0.0.23.post1',
        'Forge': '0.0.23.post1'
    }
    if Version in xformersver:
        !pip install xformers=={xformersver[Version]}
    clear_output()
    !pip show torch xformers

def install_custom():
    """Özel eklentileri yükle"""
    if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2" or Version == "ComfyUI":
        html="<p style='color:lightgreen'><b><code>{Veri Klasörü}/Setting içindeki Özel dosyasını düzenleyerek her Sürüm için isteğe bağlı extension/node ekleyebilirsiniz.</code></b></p>"
        display(HTML(html))
        print('[1;32mÖzel Extension/Node yükleniyor...')
        with capture.capture_output() as cap:
            if Version != "ComfyUI" :
                %cd {Version_folder}/extensions
                exfile = "Custom_Automatic.txt" if Version == "Automatic" else "Custom_Forgev2.txt" if Version == "Forge-v2" else "Custom_Forge.txt"
                if os.path.isfile(f"{User_folder}/Setting/{exfile}"):
                    with open(f"{User_folder}/Setting/{exfile}", "r") as file:
                        lists = file.readlines()
                for list in lists :
                    !git clone {list}
                %cd /content/SDVN
            else :
                %cd {Version_folder}/custom_nodes
                if os.path.isfile(f"{User_folder}/Setting/Custom_ComfyNode.txt"):
                    with open(f"{User_folder}/Setting/Custom_ComfyNode.txt", "r") as file:
                        notes = file.readlines()
                    for note in notes :
                        last_segment = note.rsplit('/', 1)[-1].strip()
                        if not os.path.exists(f'{last_segment }'):
                            !git clone {note}
                            !pip install -q -r {Version_folder}/custom_nodes/{last_segment}/requirements.txt
                %cd /content/ComfyUI
        inf('✔ Tamamlandı','success', '150px')

# Ana kurulum bölümü başlangıcı
%cd {root_folder}

# Gerekli klasörleri oluştur
folder_list = [ '',
                'Model',
                'Lora',
                'Embeddings',
                'wildcards',
                'Hypernetworks',
                'Export',
                'Export/ComfyUI',
                'Export/Fooocus',
                'Export/AutoRetouch',
                'ControlnetModel',
                'TrainData',
                'AnimateDiffModel',
                'Lora/AnimateDiffLora',
                'ComfyUIinput',
                'Setting',
                'Setting/Comfy_Setting',
                'Setting/Comfy_Setting/default',
                'Setting/Comfy_Setting/default/workflows',
                'ComfyModel',
                'ComfyModel/clip',
                'ComfyModel/clip_vision',
                'ComfyModel/diffusion_models',
                'ComfyModel/unet',
                'ComfyModel/diffusers',
                'ComfyModel/upscale_models',
                'ComfyModel/vae',
                'ComfyModel/inpaint',
                'ComfyModel/checkpoints',
                'ComfyModel/facerestore_models',
                'ComfyModel/ipadapter',
                'ComfyModel/instantid',
                'ComfyModel/antelopev2',
                'ComfyModel/pulid',
                'ComfyModel/style_models',
                'ComfyModel/ipadapter-flux',
                'ComfyModel/animatediff_models']
                
for i in folder_list:
    c_folder(i)

# Varsayılan dosyaları kopyala
default_file_check(f"{TSDFolder}/Setting",User_folder)

# WebUI ayarlarında yolu güncelle
file_path = f'{User_folder}/Setting/WebUI_Setting.json'
with open(file_path, 'r') as file:
    json_str = file.read()
default_path = "/content/drive/MyDrive/SD-Data"
user_path = f"{User_folder}"
json_str = json_str.replace(default_path, user_path)
with open(file_path, 'w') as file:
    file.write(json_str)
clear_output()

# Başlangıç mesajı ve resim göster
html="<h1 style='color:lightgreen; font-size:3em' ><code>TSD-WebUI KURULUMU BAŞLIYOR - LÜTFEN BEKLEYİN...</code></h1><h3><code>☕️ Bir kahve alın, 5-10 dakika bekleyin...</code></h3><h3><code>☕️ Veri klasörünüzdeki Setting klasöründeki txt dosyaları üzerinden özel Extension/Node yükleyebilirsiniz</code></h3><p>_________</p>"
display(HTML(html))
image_url = 'https://github.com/projeman/TSD-WebUI/blob/main/docs/kurulum.jpg?raw=true'
response = requests.get(image_url)
display(Image(data=response.content))

# Önceden tanımlı ayar presetleri
Preset = {
    "Comfy_SD_hizli(2dk)":[True,"ComfyUI","base","base","RealisticVision51","sd_defaultGraph.js"],
    "Comfy_flux_hizli(3dk)":[True,"ComfyUI","base","base","Flux_dev_v1-fp8","flux_defaultGraph.js"],
    "Comfy_flux_temel(7dk)":[False,"ComfyUI","base","base","Flux_dev_v1-fp8","flux_defaultGraph.js"],
    "Comfy_flux_tam(9dk)":[False,"ComfyUI","full","base","Flux_dev_v1-fp8","flux_defaultGraph.js"],
    "Comfy_flux_foto(12dk)":[False,"ComfyUI","+photo","base","Flux_dev_v1-fp8","flux_defaultGraph.js"],
    "Forge_flux_hizli(5dk)":[True,"Forge-v2","base","base","Flux_dev_v1-nf4","flux_defaultGraph.js"],
    "Forge_flux_temel(9dk)":[False,"Forge-v2","base","base","Flux_dev_v1-nf4","flux_defaultGraph.js"],
    "Automatic_SD_hizli(5dk)":[True,"Automatic","base","base","RealisticVision51","sd_defaultGraph.js"],
    "Automatic_SD_temel(9dk)":[False,"Automatic","base","base","","sd_defaultGraph.js"],
    "Automatic_SD_tam(11dk)":[False,"Automatic","full","base","","sd_defaultGraph.js"],
    "Automatic_SD_foto(12dk)":[False,"Automatic","+photo","base","","sd_defaultGraph.js"],
    "ComfyWf_Oto_YuzDegistir":[False,"ComfyUI","base","none","AdamXL-v3","Auto_SwapFace_Graph.js"],
}

# Preset ayarlarını uygula
var = ['FastMode','Version','OptionMode','Controlnet','SDmodel','my_default_name']
if Setting_Preset != 'None':
    for i in var:
        globals()[i] = Preset[Setting_Preset][var.index(i)]
        
Lib = True

# Hızlı mod ayarları
if FastMode:
    Controlnet = "none"
    Lib = False

# ControlNet modellerini indir
if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2" or Version == "ComfyUI":
    controlnetpath = f"{TSDFolder}/ControlNet"
    list_controlnet = []
    
    if 'https' in Controlnet:
        list_controlnet += Controlnet.split(',')
        Controlnet = 'none'
        
    if OptionMode == '+mhd':
        list_controlnet += ['https://huggingface.co/StableDiffusionVN/XLControlnet/resolve/main/xinsir/xinsir_union_promax-sdxl.safetensors']
        
    if Setting_Preset == 'ComfyWf_Oto_YuzDegistir':
        list_controlnet += ['https://huggingface.co/StableDiffusionVN/Basecontrolnet/resolve/main/InstantID/control_instant_id_sdxl.safetensors']
        
    for i in list_controlnet:
        download_controlnet(i)

    if Controlnet != "none":
        if OptionMode == "base":
            download_txt_controlnet('base.txt')
        else:
            download_txt_controlnet('base_full.txt')
    if Controlnet == "+SD15":
        download_txt_controlnet('+SD15.txt')
    if Controlnet == "+SDXL":
        download_txt_controlnet('+SDXL.txt')
    if Controlnet == "+Flux":
        download_txt_controlnet('+Flux.txt')
    if Controlnet == "User_list":
        download_txt_controlnet(f'{User_folder}/Setting/Custom_User_Controlnet_list.txt')
        
    link_folder(f'{User_folder}/ControlnetModel',f'{controlnetpath}/ControlnetModel')

    # Kütüphane dosyalarını indir
    if Lib == True :
        !git clone https://github.com/StableDiffusionVN/WC-SDVN
        link_folder(f'{root_folder}/WC-SDVN',f'{User_folder}/wildcards/WC-SDVN')
        !git clone https://huggingface.co/StableDiffusionVN/clone
        link_folder(f'{root_folder}/clone/Negative',f'{User_folder}/Embeddings/Negative')
        link_folder(f'{root_folder}/clone/Lora',f'{User_folder}/Lora/Lora')

with capture.capture_output() as cap:
    # WebUI kurulumu
    if Version =="Forge" or Version =="Automatic" or Version =="Forge-v2":
        !pip install -U wandb==0.16.0
        Version_folder = f"{root_folder}/SDVN"
        if test != Version:
            !rm -rf /content/SDVN
        test = Version

        if Version == "Automatic":
            !git clone https://github.com/phamhungd/SDVN
        if Version == "Forge" :
            !git clone -b F-19 https://github.com/phamhungd/SDVN-Forge /content/SDVN
        if Version == "Forge-v2":
            !git clone https://github.com/lllyasviel/stable-diffusion-webui-forge /content/SDVN

        if Version != "Forge-v2":
            Auto_ex = "TSD-WebUI/Extension/Automatic"
            if FastMode:
                run_list_txt(f"{Auto_ex}/ex_fast.txt")
            else:
                run_list_txt(f"{Auto_ex}/{'ex_base.txt' if OptionMode == 'base' else 'ex_full.txt'}")
                if OptionMode !="base" and OptionMode != "full":
                    exlist = {'+photo':'ex_photo.txt','+video':'ex_video.txt','+dev':'ex_dev.txt'}
                    if OptionMode in exlist:
                        run_list_txt(f"{Auto_ex}/{exlist[OptionMode]}")
        elif not FastMode:
            run_list_txt(f'{TSDFolder}/Extension/Forgev2/forge_ex.txt')

        link_folder(controlnetpath,f'{Version_folder}/models/ControlNet')
        aria_down('https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt',f'{Version_folder}/models/VAE','VAE84.vae.pt')
        aria_down('https://huggingface.co/phamhungd/GuoZovya/resolve/main/4x-UltraSharp.ckpt',f'{Version_folder}/models/ESRGAN','4x-UltraSharp.pth')
        link_folder(f'{User_folder}/wildcards',f'{Version_folder}/extensions/Dynamic-prompts/wildcards')
        default_file_check(f"{TSDFolder}/SDVN",root_folder,True)

    # ComfyUI kurulumu
    if Version == "ComfyUI":
        time = datetime.now().date()
        !mkdir -p {User_folder}/Export/ComfyUI/{time}

        %cd {root_folder}
        Version_folder = f"{root_folder}/ComfyUI"
        !git clone https://github.com/comfyanonymous/ComfyUI
        !pip install -q -r /content/ComfyUI/requirements.txt
        default_file_check(f"{TSDFolder}/templates",frontend_folder,True)

        %cd {Version_folder}/custom_nodes
        comfy_node = f"{TSDFolder}/Extension/Comfy"
        if FastMode:
            run_list_txt(f'{comfy_node}/node_fast.txt')
        else:
            run_list_txt(f"{comfy_node}/{'node_base.txt' if OptionMode == 'base' else 'node_full.txt'}")
            if OptionMode !="base" and OptionMode != "full":
                exlist = {'+photo':'node_photo.txt','+video':'node_video.txt','+train':'node_dev.txt','+mhd':'node_mhd.txt'}
                if OptionMode in exlist:
                    run_list_txt(f"{comfy_node}/{exlist[OptionMode]}")

        %cd {root_folder}

        # Klasör bağlantıları oluştur
        list_link_folder = {
            'Model':'models/checkpoints/Model',
            'Lora':'models/loras/Lora',
            'Embeddings':'models/embeddings/Embeddings',
            'Hypernetworks':'models/hypernetworks/Hypernetworks',
            'wildcards':'wildcards',
            'AnimateDiff':'custom_nodes/ComfyUI-AnimateDiff-Evolved/models/AnimateDiff',
            'Lora/AnimateDiff':'custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/Lora',
            'Setting/Comfy_Setting':'user',
            'Setting/Comfy_Setting/default/rgthree_config.json':'custom_nodes/rgthree-comfy/rgthree_config.json',
            'Setting/Comfy_Styles.csv':'custom_nodes/SDVN_Comfy_node/my_styles.csv',
            'Setting/API_key_for_sdvn_comfy_node.json':'custom_nodes/SDVN_Comfy_node/API_key.json',
            f'Export/ComfyUI/{time}':f'output',
        }

        !mkdir -p  {Version_folder}/styles

        for key, value in list_link_folder.items():
            link_folder(f'{User_folder}/{key}',f'{Version_folder}/{value}')
        link_folder(controlnetpath,f'{Version_folder}/models/controlnet')
        
        if Setting_Preset == 'None':
            my_default_name = 'sd_defaultGraph.js'
        defaultGraph(my_default_name,'defaultGraph.js')
        
        !cp -f {TSDFolder}/ComfySetting/favicon.ico {Version_folder}/custom_nodes/ComfyUI-Custom-Scripts/web/js/assets
        !cp -f {TSDFolder}/ComfySetting/favicon-active.ico {Version_folder}/custom_nodes/ComfyUI-Custom-Scripts/web/js/assets
        !cp -f {TSDFolder}/ComfySetting/extra_model_paths.yaml {Version_folder}
        replace_text_file(f'{Version_folder}/extra_model_paths.yaml', '/content/drive/MyDrive/SD-Data', User_folder)
        
    # Kohya kurulumu
    if Version == "Kohya" :
        Version_folder = f"{root_folder}/KohyaUI"
        %cd {root_folder}
        !git clone -b v24.1.4 https://github.com/phamhungd/KohyaUI
        %cd {Version_folder}
        !./setup.sh
        
    # Fooocus kurulumu
    if Version =="Fooocus" :
        Version_folder = f"{root_folder}/Fooocus"
        %cd {root_folder}
        !git clone https://github.com/lllyasviel/Fooocus
        %cd {Version_folder}
        link_folder(f'{User_folder}/Export/Fooocus',f'{Version_folder}/outputs')
        link_folder(f'{User_folder}/Lora',f'{Version_folder}/models/loras')
        !cp -f {TSDFolder}/FooocusSetting/Fooocus_config.txt {Version_folder}/config.json
        
    # AutoRetouch kurulumu
    if Version == "AutoRetouch" :
        Version_folder = f"{root_folder}/AutoRetouch"
        !git clone https://github.com/phamhungd/auto_retoucher {Version_folder}
        !pip install -q -r {Version_folder}/requirements.txt
        link_folder(f'{User_folder}/Export/AutoRetouch',f'{Version_folder}/outputs')
        
    # FluxGym kurulumu
    if Version == "FluxGym" :
        Version_folder = f"{root_folder}/fluxgym"
        %cd /content
        !git clone https://github.com/cocktailpeanut/fluxgym
        !git clone -b sd3 https://github.com/kohya-ss/sd-scripts {Version_folder}/sd-scripts
        %cd {Version_folder}/sd-scripts
        !pip install -q -r requirements.txt
        %cd {Version_folder}
        !pip install -q -r requirements.txt

inf('✔ Kurulum Tamamlandı','success', '150px')

# Model indirme bölümü
if Version =="Forge" or Version =="Automatic" or Version == "Forge-v2":
    Version_path =f"{Version_folder}/models/Stable-diffusion/Checkpoint"
if Version == "ComfyUI" :
    Version_path = f"{Version_folder}/models/checkpoints/Checkpoint"
if Version == "Fooocus" :
    Version_path = f"{Version_folder}/models/checkpoints"
if Version == "AutoRetouch" :
    Version_path = f"{Version_folder}/models"

checkpointpatch = f"{root_folder}/Checkpoint"
os.makedirs(checkpointpatch, exist_ok=True)

# Model kütüphanesini yükle
file_path = f'{TSDFolder}/model_lib.json'
with open(file_path, 'r') as json_file:
    modellist = json.load(json_file)

# Hızlı modda varsayılan model indir
if FastMode and SDmodel == "":
    SDmodel = 'https://huggingface.co/StableDiffusionVN/Checkpoint/resolve/main/SD15/Real/RealisticVision-v5.safetensors@=RealisticVision-v5'
    
# Model listesini işle
list_download = ['SDmodel','SD15model','SDXLmodel','Fluxmodel','TSDmodel','InpaintMd']
for i in list_download:
    download_lib(globals()[i],modellist)

# Özel model listesini indir
if not FastMode and Version != "FluxGym" and "ComfyWf" not in Setting_Preset:
    file_path = f'{User_folder}/Setting/Custom_Model_List.txt'
    with open(file_path, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]
    download_txt(lines)
    
if Version != "Kohya" and Version != "FluxGym":
    link_folder(f'{checkpointpatch}',f'{Version_path}')
    
print("\n🎉 TSD-WebUI kurulumu başarıyla tamamlandı!")
print("🇹🇷 Türk yapımı AI aracı - Made with ❤️ in Turkey")